In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json
import optuna
from optuna.samplers import TPESampler

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, BatchNormalization, Input, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.regularizers import l2

In [ ]:
# Unzip dataset in Google Colab
import os
import zipfile
import shutil

zip_file_path = "/content/cats-v-non-cats.zip"
extract_dir = "/content/cats-v-non-cats/"

os.makedirs(extract_dir, exist_ok=True)

try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        for file_info in zip_ref.infolist():
            # Extract only the files we want, avoiding the extra directory structure and __MACOSX
            if not file_info.filename.startswith('__MACOSX/'):
                # Remove the leading 'cats-v-dogs/' from the filename if it exists
                arcname = file_info.filename
                if arcname.startswith('cats-v-non-cats/'):
                    arcname = arcname[len('cats-v-non-cats/'):]

                # Only extract if the filename is not empty after removing the prefix
                if arcname:
                    file_info.filename = arcname
                    zip_ref.extract(file_info, extract_dir)


    print(f"File extracted to: {extract_dir}")

    # Remove the __MACOSX folder if it exists
    macosx_folder = os.path.join("/content/", '__MACOSX')
    if os.path.exists(macosx_folder):
        shutil.rmtree(macosx_folder)
        print(f"Folder '{macosx_folder}' deleted successfully.")

except Exception as e:
    print(f"An error occurred during unzipping: {e}")

In [ ]:
# Configuration and Data Paths
TRAINING_DIR = "cats-v-non-cats/training/"
VALIDATION_DIR = "cats-v-non-cats/validation/"
TESTING_DIR = "cats-v-non-cats/test/"

# Define whether to include test split or not
INCLUDE_TEST = True

# Baseline Configuration for comparison
BASELINE_CONFIG = {
    'learning_rate': 0.001,
    'reg_strength': 0.00001,
    'dropout_conv': 0.15,
    'dropout_dense': 0.4,
    'dense_units': 512,
    'filters_multiplier': 0.75,
    'batch_size': 32,
    'beta_1': 0.8,
    'beta_2': 0.99
}

In [ ]:
# Set up data generators
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

validation_gen = ImageDataGenerator(rescale=1./255)

if INCLUDE_TEST:
    test_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Model Architecture Function
def create_tuned_model(reg_strength=0.0001, dropout_conv=0.2, dropout_dense=0.4,
                      dense_units=512, filters_multiplier=1):

    inputs = Input(shape=(128, 128, 3))

    # Calculate filter sizes
    filters1 = int(32 * filters_multiplier)
    filters2 = int(64 * filters_multiplier)
    filters3 = int(128 * filters_multiplier)

    # First block
    x = Conv2D(filters1, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(reg_strength))(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(filters1, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(reg_strength))(x)
    x = MaxPooling2D(2, 2)(x)
    x = Dropout(dropout_conv)(x)

    # Second block
    x = Conv2D(filters2, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(reg_strength))(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters2, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(reg_strength))(x)
    x = MaxPooling2D(2, 2)(x)
    x = Dropout(dropout_conv)(x)

    # Third block
    x = Conv2D(filters3, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(reg_strength))(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters3, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(reg_strength))(x)
    x = MaxPooling2D(2, 2)(x)
    x = Dropout(dropout_conv)(x)

    # Global pooling and dense layers
    x = GlobalAveragePooling2D()(x)
    x = Dense(dense_units, activation='relu', kernel_regularizer=l2(reg_strength))(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_dense)(x)
    x = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg_strength))(x)

    return Model(inputs=inputs, outputs=x)

In [ ]:
# Training and Evaluation Function
def train_and_evaluate_model(config, epochs=5, param_name=None, param_value=None):
    try:
        # Create model with current configuration
        model = create_tuned_model(
            reg_strength=config['reg_strength'],
            dropout_conv=config['dropout_conv'],
            dropout_dense=config['dropout_dense'],
            dense_units=config['dense_units'],
            filters_multiplier=config['filters_multiplier']
        )

        # Compile model with current optimizer settings
        optimizer = Adam(
            learning_rate=config['learning_rate'],
            beta_1=config['beta_1'],
            beta_2=config['beta_2']
        )

        model.compile(
            optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy', 'AUC']
        )

        train_generator = train_gen.flow_from_directory(
            TRAINING_DIR,
            target_size=(128, 128),
            batch_size=config['batch_size'],
            class_mode='binary',
            shuffle=True
        )

        validation_generator = validation_gen.flow_from_directory(
            VALIDATION_DIR,
            target_size=(128, 128),
            batch_size=config['batch_size'],
            class_mode='binary',
            shuffle=False
        )

        if INCLUDE_TEST:
            test_generator = test_gen.flow_from_directory(
                TESTING_DIR,
                target_size=(128, 128),
                batch_size=config['batch_size'],
                class_mode='binary',
                shuffle=True
            )

        # Define callbacks for training (no checkpoint saving)
        reduce_lr = ReduceLROnPlateau(
            monitor='val_accuracy',
            factor=0.2,
            patience=5,
            min_lr=1e-8,
            verbose=1
        )

        early_stop = EarlyStopping(
            monitor='val_accuracy',
            patience=15,
            restore_best_weights=True,
            verbose=1,
            mode='max'
        )

        # Only use learning rate reduction and early stopping - no checkpoint saving
        callbacks = [reduce_lr, early_stop]

        # Train model
        history = model.fit(
            train_generator,
            validation_data=validation_generator,
            epochs=epochs,
            verbose=1,
            callbacks=callbacks
        )

        # Get final validation metrics
        val_loss, val_accuracy, val_auc = model.evaluate(validation_generator, verbose=0)

        # Get test metrics if test set is available
        test_results = {}
        if INCLUDE_TEST:
            test_loss, test_accuracy, test_auc = model.evaluate(test_generator, verbose=0)
            test_results = {
                'test_accuracy': test_accuracy,
                'test_auc': test_auc,
                'test_loss': test_loss
            }

        results = {
            'val_accuracy': val_accuracy,
            'val_auc': val_auc,
            'val_loss': val_loss,
            'history': history.history,
            'batch_size_used': config['batch_size']
        }

        # Add test results if available
        results.update(test_results)

        return results

    except Exception as e:
        print(f"Error in training: {e}")
        return {
            'val_accuracy': 0.0,
            'val_auc': 0.0,
            'val_loss': float('inf'),
            'error': str(e)
        }

In [ ]:
# Bayesian Optimization Setup
def objective(trial):
    config = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
        'reg_strength': trial.suggest_float('reg_strength', 1e-6, 1e-2, log=True),
        'dropout_conv': trial.suggest_float('dropout_conv', 0.1, 0.4),
        'dropout_dense': trial.suggest_float('dropout_dense', 0.2, 0.7),
        'dense_units': trial.suggest_categorical('dense_units', [128, 256, 512, 1024, 2048]),
        'filters_multiplier': trial.suggest_float('filters_multiplier', 0.5, 2.0),
        'batch_size': trial.suggest_categorical('batch_size', [16, 32, 64, 128]),
        'beta_1': trial.suggest_float('beta_1', 0.7, 0.99),
        'beta_2': trial.suggest_float('beta_2', 0.9, 0.9999)
    }

    try:
        result = train_and_evaluate_model(
            config,
            epochs=30,
            param_name=f"trial_{trial.number}",
            param_value="bayesian"
        )
        accuracy = result['val_accuracy']

        trial.set_user_attr('val_auc', result['val_auc'])
        trial.set_user_attr('val_loss', result['val_loss'])

        print(f"Trial {trial.number}: Accuracy = {accuracy:.4f}")

        return accuracy

    except Exception as e:
        print(f"Trial {trial.number} failed: {e}")
        # Return a low value for failed trials
        return 0.0

def run_bayesian_optimization(n_trials=50, timeout=None):

    print(f"Starting Bayesian Optimization with {n_trials} trials...")

    # Create study object
    study = optuna.create_study(
        direction='maximize',  # We want to maximize accuracy
        sampler=TPESampler(seed=42),  # Tree-structured Parzen Estimator
        study_name='cnn_hyperparameter_optimization'
    )

    # Run optimization
    study.optimize(objective, n_trials=n_trials, timeout=timeout)

    # Print results
    print(f"\n{'='*60}")
    print("BAYESIAN OPTIMIZATION RESULTS")
    print(f"{'='*60}")

    print(f"Number of finished trials: {len(study.trials)}")
    print(f"Best trial number: {study.best_trial.number}")
    print(f"Best validation accuracy: {study.best_value:.4f}")

    print(f"\n🏆 Best hyperparameters:")
    for key, value in study.best_params.items():
        print(f"  {key}: {value}")

    # Get additional metrics for best trial
    best_trial = study.best_trial
    if 'val_auc' in best_trial.user_attrs:
        print(f"\nBest trial metrics:")
        print(f"  Validation AUC: {best_trial.user_attrs['val_auc']:.4f}")
        print(f"  Validation Loss: {best_trial.user_attrs['val_loss']:.4f}")

    return study

In [ ]:
print("Training baseline model...")
baseline_result = train_and_evaluate_model(BASELINE_CONFIG, epochs=40)

print("\nBaseline Results:")
print(f"Accuracy: {baseline_result['val_accuracy']:.4f}")
print(f"AUC: {baseline_result['val_auc']:.4f}")
print(f"Loss: {baseline_result['val_loss']:.4f}")

In [ ]:
study = run_bayesian_optimization(n_trials=50)  # Increase to 100-200 for production

In [ ]:
# Analyze and Visualize Results
def analyze_bayesian_results(study):
    # Train final model with best parameters
    print(f"\n{'='*60}")
    print("🔥 TRAINING FINAL MODEL WITH BEST PARAMETERS")
    print(f"{'='*60}")

    best_config = study.best_params

    # Train with more epochs for final model
    final_result = train_and_evaluate_model(
        best_config,
        epochs=60,  # More epochs for final training
        param_name="bayesian_best",
        param_value="final"
    )

    print(f"\n📊 FINAL RESULTS COMPARISON:")
    print(f"{'='*50}")
    print(f"Baseline  - Accuracy: {baseline_result['val_accuracy']:.4f}, AUC: {baseline_result['val_auc']:.4f}")
    print(f"Bayesian  - Accuracy: {final_result['val_accuracy']:.4f}, AUC: {final_result['val_auc']:.4f}")
    print(f"Improvement - Accuracy: {final_result['val_accuracy'] - baseline_result['val_accuracy']:+.4f}, AUC: {final_result['val_auc'] - baseline_result['val_auc']:+.4f}")

    # Plot optimization history
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # 1. Optimization history
    trial_numbers = [trial.number for trial in study.trials]
    trial_values = [trial.value if trial.value is not None else 0 for trial in study.trials]

    axes[0, 0].plot(trial_numbers, trial_values, 'b-', alpha=0.6)
    axes[0, 0].scatter(trial_numbers, trial_values, c=trial_values, cmap='viridis', s=30)
    axes[0, 0].axhline(y=baseline_result['val_accuracy'], color='red', linestyle='--',
                       label=f'Baseline ({baseline_result["val_accuracy"]:.4f})')
    axes[0, 0].set_xlabel('Trial Number')
    axes[0, 0].set_ylabel('Validation Accuracy')
    axes[0, 0].set_title('Optimization History')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # 2. Parameter importance (if available)
    try:
        importance = optuna.importance.get_param_importances(study)
        params = list(importance.keys())
        values = list(importance.values())

        axes[0, 1].barh(params, values)
        axes[0, 1].set_xlabel('Importance')
        axes[0, 1].set_title('Hyperparameter Importance')
        axes[0, 1].grid(True, alpha=0.3)
    except:
        axes[0, 1].text(0.5, 0.5, 'Parameter importance\nnot available\n(need more trials)',
                        ha='center', va='center', transform=axes[0, 1].transAxes)
        axes[0, 1].set_title('Hyperparameter Importance')

    # 3. Best vs worst trials comparison
    best_trials = sorted(study.trials, key=lambda t: t.value if t.value else 0, reverse=True)[:5]
    worst_trials = sorted(study.trials, key=lambda t: t.value if t.value else 0)[:5]

    best_values = [t.value for t in best_trials if t.value is not None]
    worst_values = [t.value for t in worst_trials if t.value is not None]

    axes[1, 0].bar(range(len(best_values)), best_values, color='green', alpha=0.7, label='Best 5 trials')
    axes[1, 0].bar(range(len(best_values), len(best_values) + len(worst_values)),
                   worst_values, color='red', alpha=0.7, label='Worst 5 trials')
    axes[1, 0].set_ylabel('Validation Accuracy')
    axes[1, 0].set_title('Best vs Worst Trials')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # 4. Learning rate vs accuracy scatter
    lr_values = []
    acc_values = []
    for trial in study.trials:
        if trial.value is not None and 'learning_rate' in trial.params:
            lr_values.append(trial.params['learning_rate'])
            acc_values.append(trial.value)

    if lr_values:
        axes[1, 1].scatter(lr_values, acc_values, alpha=0.6, c=acc_values, cmap='viridis')
        axes[1, 1].set_xscale('log')
        axes[1, 1].set_xlabel('Learning Rate')
        axes[1, 1].set_ylabel('Validation Accuracy')
        axes[1, 1].set_title('Learning Rate vs Accuracy')
        axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('bayesian_optimization_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

    return final_result, best_config

# Run analysis
if 'study' in locals():
    bayesian_final_result, bayesian_best_config = analyze_bayesian_results(study)
    print("✅ Analysis complete!")
else:
    print("❌ Run Bayesian optimization first!")

In [ ]:
# Final Summary and Model Comparison
def final_summary():
    print(f"\n{'='*70}")
    print("🎯 BAYESIAN OPTIMIZATION SUMMARY")
    print(f"{'='*70}")

    if 'bayesian_final_result' in locals():
        baseline_acc = baseline_result['val_accuracy']
        optimized_acc = bayesian_final_result['val_accuracy']
        improvement = optimized_acc - baseline_acc

        print(f"📈 Performance Improvement:")
        print(f"  Baseline Accuracy:  {baseline_acc:.4f}")
        print(f"  Optimized Accuracy: {optimized_acc:.4f}")
        print(f"  Improvement:        {improvement:+.4f} ({improvement/baseline_acc*100:+.2f}%)")

        print(f"\n🏆 Best Configuration Found:")
        for param, value in bayesian_best_config.items():
            baseline_val = BASELINE_CONFIG.get(param, "N/A")
            print(f"  {param:<18}: {value:<10} (baseline: {baseline_val})")

        print(f"\n🔍 Key Insights:")
        print(f"  • Total trials run: {len(study.trials)}")
        print(f"  • Best trial: #{study.best_trial.number}")
        print(f"  • Search space explored efficiently using Bayesian optimization")
        print(f"  • Model saved as: {bayesian_final_result['model_filename']}")

        # Determine if optimization was successful
        if improvement > 0.01:  # 1% improvement threshold
            print(f"\n✅ Optimization SUCCESSFUL! Significant improvement achieved.")
        elif improvement > 0:
            print(f"\n✅ Optimization successful with modest improvement.")
        else:
            print(f"\n⚠️  Baseline was already quite good. Consider:")
            print(f"     • Running more trials")
            print(f"     • Expanding search space")
            print(f"     • Different optimization objectives")

    else:
        print("❌ Bayesian optimization results not available.")

    print(f"\n🚀 Next Steps:")
    print(f"  • Use the best model for production")
    print(f"  • Consider ensemble methods")
    print(f"  • Test on holdout data")
    print(f"  • Monitor performance in production")

# Run final summary
final_summary()